<div style="background-color:#5D73F2; color:#19180F; font-size:40px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">[Beginner Friendly] Multiple approaches </div>
<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
   <div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌     1. <b> TFIDF cosine similarity approach </b>:This approach combines the use of Term Frequency-Inverse Document Frequency (TFIDF) vectorization for text representation and cosine similarity to find similar questions in the training set.<br><br>
📌      2. <b> TFIDF-NN approach </b>:This approach combines the use of Term Frequency-Inverse Document Frequency (TFIDF) vectorization for text representation and a Nearest Neighbors model to find similar questions in the training set. By encoding the labels as integers, it calculates the most frequent answers within the nearest neighbors to generate predictions for the test set. The simple ensemble strategy helps in generating the submission for the Kaggle-LLM Science Exam competition.<br><br>
📌      3. <b> TFIDF-RF approach </b>:This approach combines the use of Term Frequency-Inverse Document Frequency (TFIDF) vectorization for text representation and a Random forest model to find similar questions in the training set.<br><br>
📌      4. <b> Using BERT </b>:This approach trains a BERT for question answering on train.csv to find similar questions in the test set.<br><br>
📌      5. <b> Using T5 </b>:This approach trains a T5 for question answering on train.csv to find similar questions in the test set.        </div>

<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
📌
       1. <b> TFIDF-Cosine similarity approach </b>:</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules    </div>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data    </div>

In [2]:
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Preprocessing the data    </div>

In [3]:
vectorizer = TfidfVectorizer(stop_words='english')
train_vectors = vectorizer.fit_transform(train_df['prompt'])
test_vectors = vectorizer.transform(test_df['prompt'])



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Calculating cosine similarity between test prompts and train prompts    </div>

In [4]:
cosine_similarities = cosine_similarity(test_vectors, train_vectors)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generate predictions </div>

In [5]:
num_predictions = 3
prediction_ids = []
prediction_labels = []


In [6]:
for i, cos_sim in enumerate(cosine_similarities):
    top_indices = cos_sim.argsort()[-num_predictions:][::-1]
    top_labels = [train_df.loc[idx, 'answer'] for idx in top_indices]
    
    prediction_ids.extend([i] * num_predictions)
    prediction_labels.extend(top_labels)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Create submission dataframe    </div>

In [7]:
submission_df = pd.DataFrame({'id': prediction_ids, 'prediction': prediction_labels})



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Format predictions as per requirement    </div>

In [8]:
submission_df['prediction'] = submission_df.groupby('id')['prediction'].transform(lambda x: ' '.join(x))
submission_df = submission_df.drop_duplicates(subset='id')



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Save submission to csv    </div>

In [9]:
# Save submission to a CSV file
submission_df.to_csv('submission.csv', index=False)

<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
📌
       2. <b> TFIDF-NN approach </b>:</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules    </div>

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the train and test data    </div>

In [11]:
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Seperating the features and labels    </div>

In [12]:
X_train = train_df['prompt']
y_train_labels = train_df[['A', 'B', 'C', 'D', 'E']].values
y_train = np.argmax(y_train_labels, axis=1)  # Encode labels as integers (0, 1, 2, 3, 4)
X_test = test_df['prompt']


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Fitting the model    </div>

In [13]:
# Convert text data to numerical vectors using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Build Nearest Neighbors model
k_neighbors = 10
nn_model = NearestNeighbors(n_neighbors=k_neighbors)
nn_model.fit(X_train_vect)

NearestNeighbors(n_neighbors=10)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Function to predict answers    </div>

In [14]:
def predict_answers(model, X_test, y_train, k=3):
    top_k_indices = model.kneighbors(X_test, return_distance=False)
    predictions = []
    for indices in top_k_indices:
        top_k_labels = y_train[indices]  
        top_k_labels_counts = np.bincount(top_k_labels)
        top_k_labels_sorted = np.argsort(top_k_labels_counts)[::-1]
        top_k_labels_sorted = [chr(65 + label) for label in top_k_labels_sorted]
        predictions.append(" ".join(top_k_labels_sorted[:k]))
    return predictions

# Make predictions for the test set
predictions = predict_answers(nn_model, X_test_vect, y_train)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating submission    </div>

In [15]:
submission_df = pd.DataFrame({'id': test_df.index, 'prediction': predictions})
submission_df.to_csv('submission.csv', index=False)


In [16]:
submission_df

,id,prediction
0,0,B A E
1,1,C B A
2,2,B A D
3,3,E D C
4,4,A D C
...,...,...
195,195,D A E
196,196,E C A
197,197,A D C
198,198,D A B


<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
📌
       3. <b> TFIDF-RF approach </b>:</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules   </div>

In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data and seperating into features and labels   </div>

In [18]:
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
X_train = train_df['prompt']
y_train_labels = train_df[['A', 'B', 'C', 'D', 'E']].values
y_train = np.argmax(y_train_labels, axis=1)  # Encode labels as integers (0, 1, 2, 3, 4)
X_test = test_df['prompt']



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Convert text data into numerical vectors using TfidfVectorizer   </div>

In [19]:
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Function to predict answers   </div>

In [20]:
def predict_answers(model, X_test, k=3):
    y_pred_probs = model.predict_proba(X_test)
    top_k_indices = np.argsort(-y_pred_probs, axis=1)[:, :k]  # Get indices of top k predictions
    predictions = []
    for indices in top_k_indices:
        top_k_labels = indices
        top_k_labels_counts = np.bincount(top_k_labels)
        top_k_labels_sorted = np.argsort(top_k_labels_counts)[::-1]
        top_k_labels_sorted = [chr(65 + label) for label in top_k_labels_sorted]
        predictions.append(" ".join(top_k_labels_sorted[:k]))
    return predictions


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Spliit the data into train and val sets and train random forest classifier over 5 folds   </div>

In [21]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds_predictions = []

for train_index, val_index in skf.split(X_train_vect, y_train):
    X_train_fold, X_val_fold = X_train_vect[train_index], X_train_vect[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model (you can experiment with other advanced models here)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    val_predictions = predict_answers(model, X_val_fold)
    folds_predictions.append(val_predictions)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Choose the most common prediction among all folds as final val preds   </div>

In [22]:
val_final_predictions = []
for i in range(len(y_val_fold)):
    labels = [fold_preds[i].split() for fold_preds in folds_predictions]
    labels_flattened = [label for sublist in labels for label in sublist]
    val_final_predictions.append(" ".join(labels_flattened[:3]))



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Calculating map@3 on val set  </div>

In [23]:
def mapk(y_true, y_pred, k=3):
    assert len(y_true) == len(y_pred), "Number of samples in y_true and y_pred must be the same."
    total_mapk = 0
    for true_labels, pred_labels in zip(y_true, y_pred):
        # Convert sets to lists
        true_labels_list = list(true_labels)
        pred_labels_list = list(pred_labels)
        # Get the top-k predicted labels
        pred_labels_top_k = pred_labels_list[:k]
        # Compute average precision for the current sample
        mapk = sum([1 / (i + 1) for i, label in enumerate(pred_labels_top_k) if label in true_labels_list]) / min(len(true_labels_list), k)
        total_mapk += mapk
    return total_mapk / len(y_true)

val_map_3 = mapk([set([label]) for label in y_val_fold], [set(pred.split()) for pred in val_final_predictions])
print(f"Validation MAP@3: {val_map_3}")


Validation MAP@3: 0.0



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Train model on the entire train set   </div>

In [24]:
final_model = RandomForestClassifier(n_estimators=100, random_state=42)
final_model.fit(X_train_vect, y_train)


RandomForestClassifier(random_state=42)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Make predictions on the test set   </div>

In [25]:
test_predictions = predict_answers(final_model, X_test_vect)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Save predictions to a submission file   </div>

In [26]:
submission_df = pd.DataFrame({'id': test_df.index, 'prediction': test_predictions})
submission_df.to_csv('submission.csv', index=False)

In [27]:
submission_df

,id,prediction
0,0,D B A
1,1,C B A
2,2,C B A
3,3,E C B
4,4,D C B
...,...,...
195,195,E C A
196,196,E D A
197,197,D B A
198,198,C B A


<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
📌
<b> Using BERT </b>:
<br>
📌 To know more about BERT, Visit the <a href="https://www.kaggle.com/code/suraj520/bert-know-fit-infer"> kernel </a>
</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules  </div>

In [28]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining the BERT model class   </div>

In [29]:
class BertForQuestionAnswering(nn.Module):
    def __init__(self, bert_model):
        super(BertForQuestionAnswering, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, 5)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.last_hidden_state[:, 0, :])
        return logits


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data   </div>

In [30]:
train_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Initializing the BERT tokenizer and model   </div>

In [31]:
tokenizer = BertTokenizer.from_pretrained("/kaggle/input/huggingface-bert/bert-large-cased")
bert_model = BertModel.from_pretrained("/kaggle/input/huggingface-bert/bert-large-cased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForQuestionAnswering(bert_model).to(device)


Some weights of the model checkpoint at /kaggle/input/huggingface-bert/bert-large-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining dataset class   </div>

In [32]:
class CustomDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=128, mode="train"):
        self.data_df = data_df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mode=mode

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        prompt = row['prompt']
        options = [row['A'], row['B'], row['C'], row['D'], row['E']]
        if self.mode=="train":
            answer = row['answer']

        inputs = self.tokenizer(prompt, options, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        # Map the correct answer label to its index
        if self.mode=="train":
            label_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
            label_idx = label_mapping[answer]

            return input_ids, attention_mask, label_idx  # Return input data and label index
        else:
            return input_ids, attention_mask



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating dataloaders   </div>

In [33]:
train_dataset = CustomDataset(train_df, tokenizer,mode="train")
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Performing sanity check of the dataloader  </div>

In [34]:
for batch in train_loader:
    print(batch)
    break


[tensor([[ 101, 1184, 1110,  ...,    0,    0,    0],
        [ 101, 1184, 1110,  ...,    0,    0,    0],
        [ 101, 1184, 1110,  ...,    0,    0,    0],
        ...,
        [ 101, 1184, 1110,  ...,    0,    0,    0],
        [ 101, 1184, 1110,  ...,    0,    0,    0],
        [ 101, 1184, 1110,  ...,    0,    0,    0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 1, 2, 3, 2, 1, 1, 2])]



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Train the model  </div>

In [35]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training loop   </div>

In [36]:
from tqdm import tqdm

for epoch in range(10):  # You can adjust the number of epochs
    model.train()
    for batch_input_ids, batch_attention_mask,batch_labels in tqdm(train_loader):
        optimizer.zero_grad()
        logits = model(batch_input_ids.to(device), batch_attention_mask.to(device))
        loss = criterion(logits, batch_labels.to(device))
        loss.backward()
        optimizer.step()
    print("Epoch-{}, Loss-{}".format(epoch,loss.item()))

100%|██████████| 25/25 [00:10<00:00,  2.37it/s]


Epoch-0, Loss-1.8979820013046265


100%|██████████| 25/25 [00:09<00:00,  2.77it/s]


Epoch-1, Loss-1.408605694770813


100%|██████████| 25/25 [00:09<00:00,  2.77it/s]


...

Epoch-7, Loss-0.056274015456438065


100%|██████████| 25/25 [00:09<00:00,  2.77it/s]


Epoch-8, Loss-0.0479782372713089


100%|██████████| 25/25 [00:09<00:00,  2.77it/s]


Epoch-9, Loss-0.0164398942142725



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generate predictions on test data   </div>

In [37]:
test_dataset = CustomDataset(test_df, tokenizer,mode="test")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Putting model to eval mode   </div>

In [38]:
model.eval()
predictions = []


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating predictions  </div>

In [39]:
with torch.no_grad():
    for batch_input_ids, batch_attention_mask in tqdm(test_loader):
        logits = model(batch_input_ids.to(device), batch_attention_mask.to(device))
        _, predicted = torch.topk(logits, 3, dim=1)
        predictions.extend(predicted.tolist())


100%|██████████| 7/7 [00:02<00:00,  2.59it/s]


In [40]:
predictions

[[3, 1, 4],
 [0, 3, 4],
 [0, 2, 4],
 [2, 4, 0],
 [3, 1, 0],
 [1, 0, 3],
 [0, 3, 1],
 [3, 4, 1],
 [2, 4, 0],
 [0, 4, 2],
 [4, 3, 1],
 [0, 4, 3],
 [2, 0, 1],
 [3, 4, 1],
 [1, 3, 2],
 [1, 2, 4],
 [4, 3, 1],
 [4, 3, 1],
 [0, 1, 4],
 [4, 3, 1],
 [3, 4, 0],
 [3, 4, 0],
 [2, 1, 0],
 [2, 0, 4],
 [4, 3, 0],
 [4, 3, 0],
 [0, 4, 2],
 [3, 1, 4],
 [4, 3, 2],
 [2, 0, 3],
 [1, 0, 4],
 [4, 0, 3],
 [4, 3, 0],
 [3, 4, 2],
 [2, 0, 3],
 [1, 0, 3],
 [4, 2, 0],
 [0, 4, 2],
 [4, 0, 1],
 [4, 1, 3],
 [4, 1, 3],
 [2, 1, 0],
 [1, 4, 3],
 [2, 0, 4],
 [0, 1, 2],
 [0, 4, 2],
 [1, 0, 4],
 [2, 4, 1],
 [3, 4, 0],
 [1, 4, 2],
 [1, 0, 2],
 [4, 1, 0],
 [2, 3, 0],
 [0, 1, 2],
 [1, 4, 0],
 [1, 3, 4],
 [2, 0, 4],
 [2, 4, 0],
 [3, 4, 0],
 [0, 4, 2],
 [1, 3, 0],
 [1, 4, 3],
 [2, 0, 4],
 [2, 0, 1],
 [0, 4, 1],
 [4, 3, 0],
 [2, 0, 4],
 [4, 1, 3],
 [2, 0, 4],
 [3, 1, 4],
 [2, 4, 3],
 [0, 4, 3],
 [3, 1, 4],
 [1, 4, 3],
 [3, 4, 1],
 [1, 0, 4],
 [3, 4, 0],
 [1, 3, 2],
 [2, 0, 4],
 [4, 0, 1],
 [2, 4, 3],
 [0, 4, 3],
 [1, 4, 3],
 [0,


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating submission file   </div>

In [41]:
submission_df = pd.DataFrame({'id': test_df.index, 'prediction': predictions})
submission_df['prediction'] = submission_df['prediction'].apply(lambda x: ' '.join([chr(ord('A') + idx) for idx in x]))
submission_df.to_csv('submission.csv', index=False)

In [42]:
submission_df

,id,prediction
0,0,D B E
1,1,A D E
2,2,A C E
3,3,C E A
4,4,D B A
...,...,...
195,195,C A D
196,196,B A E
197,197,B E A
198,198,D E A


<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">  
📌
<b> Using T5 </b>:
<br>
📌 To know more about T5, Visit the <a href="https://www.kaggle.com/code/suraj520/t5-multi-gpu-know-train-infer"> kernel </a>
</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules  </div>

In [43]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining T5 model class for QnA  </div>

In [44]:
class T5ForQuestionAnswering(nn.Module):
    def __init__(self, t5_model):
        super(T5ForQuestionAnswering, self).__init__()
        self.t5 = t5_model

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, labels=None):
        outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, labels=labels)
        return outputs.loss, outputs.logits


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data </div>

In [45]:
train_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")


In [46]:
# # Define paths for saving and loading
# save_path = "/kaggle/working/t5_model"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)

# # Save tokenizer and model
# tokenizer.save_pretrained(save_path)
# t5_model.save_pretrained(save_path)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the tokenizer and model </div>

In [47]:
tokenizer = T5Tokenizer.from_pretrained("/kaggle/input/t5-model")
t5_model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/t5-model")





<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Initializing the device and model  </div>

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForQuestionAnswering(t5_model).to(device)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating custom dataset class  </div>

In [49]:
class CustomDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=128, mode="train"):
        self.data_df = data_df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mode = mode

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        prompt = row['prompt']
        options = [row['A'], row['B'], row['C'], row['D'], row['E']]
        if self.mode == "train":
            answer = row['answer']

        input_text = f"question: {prompt} options: {', '.join(options)}"
        target_text = f"answer: {answer}" if self.mode == "train" else ""

        inputs = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt', return_attention_mask=True)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        if self.mode == "train":
            target = self.tokenizer(target_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt', return_attention_mask=True)
            target_ids = target['input_ids'].squeeze()
            target_attention_mask = target['attention_mask'].squeeze()
            return input_ids, attention_mask, target_ids, target_attention_mask
        else:
            return input_ids, attention_mask



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating dataset and dataloader  </div>

In [50]:
train_dataset = CustomDataset(train_df, tokenizer,mode="train")
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Performing sanity check of the dataloader </div>

In [51]:
for batch in train_loader:
    print(batch)
    break


[tensor([[  822,    10,   363,  ...,     8, 10951,     1],
        [  822,    10,   363,  ...,  5932,    19,     1],
        [  822,    10,   363,  ...,     5,     6,     1],
        ...,
        [  822,    10,   363,  ...,     0,     0,     0],
        [  822,    10,   363,  ...,  1181,    51,     1],
        [  822,    10,   363,  ...,   295,    23,     1]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), tensor([[1525,   10,  272,  ...,    0,    0,    0],
        [1525,   10,  205,  ...,    0,    0,    0],
        [1525,   10,  205,  ...,    0,    0,    0],
        ...,
        [1525,   10,  205,  ...,    0,    0,    0],
        [1525,   10,  309,  ...,    0,    0,    0],
        [1525,   10,  262,  ...,    0,    0,    0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ...


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining optimizer and loss function </div>

In [52]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training loop </div>

In [53]:
from tqdm import tqdm

for epoch in range(50):  # You can adjust the number of epochs
    model.train()
    for batch_input_ids, batch_attention_mask, batch_target_ids, batch_target_attention_mask in tqdm(train_loader):
        optimizer.zero_grad()
        loss, logits = model(input_ids=batch_input_ids.to(device), attention_mask=batch_attention_mask.to(device), decoder_input_ids=batch_target_ids.to(device), labels=batch_target_ids.to(device))
        loss.backward()
        optimizer.step()
    print("Epoch-{}, Loss-{}".format(epoch,loss.item()))

100%|██████████| 25/25 [00:02<00:00,  9.03it/s]


Epoch-0, Loss-7.996217250823975


100%|██████████| 25/25 [00:02<00:00,  9.26it/s]


Epoch-1, Loss-2.3192126750946045


100%|██████████| 25/25 [00:02<00:00,  9.20it/s]


...

Epoch-47, Loss-0.008193301036953926


100%|██████████| 25/25 [00:02<00:00,  9.29it/s]


Epoch-48, Loss-0.018991732969880104


100%|██████████| 25/25 [00:02<00:00,  9.34it/s]

Epoch-49, Loss-0.005325842648744583



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining test dataset and test loader  </div>

In [54]:
test_dataset = CustomDataset(test_df, tokenizer,mode="test")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating predictions </div>

In [55]:
model.eval()

predictions = []

with torch.no_grad():
    for batch_input_ids, batch_attention_mask, batch_target_ids, batch_target_attention_mask in tqdm(train_loader):
        optimizer.zero_grad()
        loss, logits = model(input_ids=batch_input_ids.to(device), attention_mask=batch_attention_mask.to(device), decoder_input_ids=batch_target_ids.to(device), labels=batch_target_ids.to(device))

        # Remove the extra dimension from logits
        logits = logits[:, 0, :]

        _, predicted = torch.topk(logits, 3, dim=1)
        predictions.extend(predicted.tolist())

import numpy as np

# Reshape predictions to (200, 3)
reshaped_predictions = np.array(predictions).reshape((200, 3))

print(np.shape(reshaped_predictions))


100%|██████████| 25/25 [00:01<00:00, 20.56it/s]

(200, 3)


In [56]:
predictions

[[1525, 4269, 9986],
 [1525, 18243, 4269],
 [1525, 9986, 18243],
 [1525, 18243, 9986],
 [1525, 4269, 9986],
 [1525, 9986, 18243],
 [1525, 4269, 18243],
 [1525, 4269, 18243],
 [1525, 4269, 18243],
 [1525, 18243, 4269],
 [1525, 18243, 8776],
 [1525, 8776, 18243],
 [1525, 9986, 4269],
 [1525, 4269, 18243],
 [1525, 4269, 18243],
 [1525, 18243, 4269],
 [1525, 9986, 4269],
 [1525, 4269, 18243],
 [1525, 18243, 4269],
 [1525, 18243, 4269],
 [1525, 18243, 8776],
 [1525, 18243, 9986],
 [1525, 18243, 4269],
 [1525, 18243, 4269],
 [1525, 18243, 8776],
 [1525, 18243, 9986],
 [1525, 9986, 18243],
 [1525, 18243, 9986],
 [1525, 18243, 4269],
 [1525, 18243, 9986],
 [1525, 18243, 4269],
 [1525, 4269, 18243],
 [1525, 18243, 4269],
 [1525, 18243, 8776],
 [1525, 18243, 8776],
 [1525, 18243, 9986],
 [1525, 18243, 4269],
 [1525, 9986, 4269],
 [1525, 4269, 9986],
 [1525, 18243, 4269],
 [1525, 4269, 9986],
 [1525, 18243, 4269],
 [1525, 4269, 18243],
 [1525, 9986, 18243],
 [1525, 18243, 4269],
 [1525, 18243, 42


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Writing predictions to submission file </div>

In [57]:
submission_df = pd.DataFrame({'id': test_df.index, 'prediction': predictions})
submission_df['prediction'] = submission_df['prediction'].apply(lambda x: ' '.join([chr(ord('A') + idx) for idx in x]))
submission_df.to_csv('submission.csv', index=False)

In [58]:
submission_df

,id,prediction
0,0,ض ხ ❃
1,1,ض 䞄 ხ
2,2,ض ❃ 䞄
3,3,ض 䞄 ❃
4,4,ض ხ ❃
...,...,...
195,195,ض 䞄 ⊉
196,196,ض 䞄 ხ
197,197,ض ❃ 䞄
198,198,ض 䞄 ⹚
